# Rayyan SDK for Python

## Installation

After installing the package you can import it and create a `rayyan` instance as follows:

> Note: the python package is not yet available on PyPI, so you need to install it from the source code.

In [ ]:
from pprint import pprint
from rayyan import Rayyan

rayyan = Rayyan("creds.json")

The `creds.json` file should contain the following:

```json
{
    "access_token": "...",
    "refresh_token": "..."
}
```

A valid file can be downloaded by signing in to Rayyan then going to My Account.

> Note: this is not supported yet in production for all users.

## Accessing signed-in user data

The simplest SDK call is the one that retrieves the signed-in user's data. This is done by calling the `get_info` method of `User` object:

In [ ]:
from rayyan.user import User

user = User(rayyan).get_info()

# Remove request_token before printing
user.pop('request_token', None)
# Remove some other fields
user.pop('subscription', None)
# user.pop('feature_flags', None)
pprint(user)

## Retrieving the list of all reviews

In [ ]:
from rayyan.review import Review
rayyan_review = Review(rayyan)
reviews = rayyan_review.get_all()
pprint(reviews)
print(f'Found {len(reviews["owned_reviews"])} owned reviews and {len(reviews["collab_reviews"])} shared reviews.')

## Working on the first owned review

### Printing some review key information

In [ ]:
my_review = reviews['owned_reviews'][0]
# pprint(my_review)
print(f"Here is the first review in the list of owned reviews: '{my_review['title']}' created on '{my_review['created_at']}' and it has a total of {my_review['total_articles']} articles.")


### Listing the first 15 articles sorted by title

In [ ]:
review_id = my_review['rayyan_id']
result_params = {
    "start": 0,
    "length": 15,
    "order[0][column]": 5,
    "order[0][dir]": "asc"
    # ... other query parameters ...
}
review_results = rayyan_review.results(review_id, result_params)
print(f'Returned {len(review_results["data"])} record(s) matching {review_results["recordsFiltered"]} record(s) out of {review_results["recordsTotal"]} in total')


### Printing article titles and authors

In [ ]:
for index, result in enumerate(review_results['data']):
    print(f'{index+1}: {result["title"]}')
    print(f'    Authors: {", ".join(result["authors"])}')

### Including the first article

In [ ]:
my_article = review_results['data'][0]
article_id = my_article['id']
customization_plan = {
    "included": 1
}
rayyan_review.customize(review_id, article_id, customization_plan)
print(f'Included article with title: {my_article["title"]}')

### Filtering by included articles

In [ ]:
result_params = {
    "start": 0,
    "length": 20,
    "extra[mode]": "included"
}
included_results = rayyan_review.results(review_id, result_params)
print(f'Returned {len(included_results["data"])} record(s) matching {included_results["recordsFiltered"]} record(s) out of {included_results["recordsTotal"]} in total:')
for index, result in enumerate(included_results['data']):
    print(f'{index+1}: {result["title"]}')
    print(f'    Authors: {", ".join(result["authors"])}')


### Excluding the next 5 articles

In [ ]:
my_articles = review_results['data'][1:6]
customization_plan = {
    "included": -1
}
for article in my_articles:
    article_id = article['id']
    print(f'Excluding article {article_id}...')
    rayyan_review.customize(review_id, article_id, customization_plan)
print(f'Excluded {len(my_articles)} articles from review {review_id}.')

### Excluding the next 3 articles in bluk

In [ ]:
my_articles = review_results['data'][7:10]
rayyan_review.bulk_customizations(review_id, "included", -1, (",").join(map(str,[a['id'] for a in my_articles])))

### Getting the inclusion counts facet for the review

In [ ]:
facet_params = {
    "facets[inclusion_counts]": "1"
}
facet_info = rayyan_review.facets(review_id, facet_params)
pprint(facet_info)

### Getting Customizations for `inclusion_decisions` and `labels` of the First 15 Articles Sorted by Title

In [ ]:
customization_params = {
    "types[]": ["inclusion_decisions", "labels"]
}

pprint(rayyan_review.get_customizations(review_id, customization_params))

### Invite Collaborator to the review

In [ ]:
from rayyan.review_access import ReviewAccess
review_access_instance = ReviewAccess(rayyan)

invite_confirmation = review_access_instance.invite(review_id, 2, ["user3@example.com"], "Collaboration on data analysis.")
pprint(invite_confirmation)


### Change Collaborator to a Viewer

In [ ]:
invite_confirmation = review_access_instance.update_access(review_id, 3, ["user3@example.com"])
pprint(invite_confirmation)


### Revoke Viewer access

In [ ]:
# invite_confirmation = review_access_instance.update_access(review_id, 3, ["user3@example.com"])
# pprint(invite_confirmation)
my_review_json = rayyan_review.get(review_id)
viewr_id = my_review_json['viewers'][0]['id']

pprint(review_access_instance.revoke(review_id, viewr_id))


### Restore Viewer access

In [ ]:
pprint(review_access_instance.restore(review_id, viewr_id))
pprint(viewr_id)

### Delete Viewer access

In [ ]:
pprint(review_access_instance.delete_access(review_id, viewr_id))

## Notes

### Create note

In [ ]:
from rayyan.notes import Notes

notes_instance = Notes(rayyan)
note_id = notes_instance.create_note(review_id, my_articles[0]['id'], text="This article needs further analysis.")
print(f'Added a note to article: {my_articles[0]["title"]} \nNote id: {note_id}')


### Update Note

In [ ]:
response = notes_instance.update_note(review_id, my_articles[0]['id'], note_id, text="UPDATED This article needs further analysis.")
print(response)


### Delete Note

In [ ]:
response = notes_instance.delete_note(review_id, my_articles[0]['id'], note_id)
print(response)


## ThirdPartyAuth

### Get ThirdPartyAuth link

In [ ]:
from rayyan.third_party_auth import ThirdPartyAuth
from rayyan.conf import MENDELEY

third_party_auth_instance = ThirdPartyAuth(rayyan, MENDELEY)
auth_link = third_party_auth_instance.get_auth_link()
pprint(auth_link)

## Highlights

### Create Highlight

In [ ]:
from rayyan.highlight import Highlight
highlight_instance = Highlight(rayyan)

highlight_data = highlight_instance.create_highlight(review_id, category_id=1, keyword="important term")
pprint(highlight_data)

### Delete Highlight

In [ ]:
highlight_data = highlight_instance.delete_highlight(review_id, 1, "important term")
pprint(highlight_data)

## Duplicates
### Trigger dedup job for the review

In [ ]:
from rayyan.duplicates import Duplicates

duplicates_instance = Duplicates(rayyan)

duplicates_instance.add_duplicate(review_id)


In [ ]:
duplicates_instance.get_duplicate(review_id,my_articles[0]['id'])


## Searches

### Get Pre-signed URL

In [ ]:
from rayyan.search import Search

search = Search(rayyan)
presigned_url_dict = search.pre_signed_url(review_id)
pprint(presigned_url_dict)

### Upload Search File

In [ ]:
import os 
file_path = "fulltext.txt"
search.upload_search_file(presigned_url_dict['key'],
                          presigned_url_dict['x-amz-credential'] + "1",
                          presigned_url_dict['x-amz-algorithm'],
                          presigned_url_dict['x-amz-date'],
                          presigned_url_dict['x-amz-signature'],
                          presigned_url_dict['policy'],
                          presigned_url_dict['success_action_status'],
                          presigned_url_dict['url'],
                          file_path
                          )

### Create Search

In [ ]:
search.create(review_id, presigned_url_dict['key'],file_path )

## Admin

### Full Texts

#### List Full Texts

In [ ]:
from rayyan.admin import Admin

admin = Admin(rayyan)

full_texts = admin.list_full_texts()
pprint(full_texts)

#### Show Full Text

In [ ]:
full_text_id = full_texts[0]['id']
response = admin.show_full_text(full_text_id)
pprint(response)

#### Delete Full Text

In [ ]:
response = admin.delete_full_texts(full_text_id)
pprint(response)

### Searches

#### List Searches

In [ ]:
searches = admin.list_searches()
pprint(searches)

#### Show Search

In [ ]:
search_id = searches[0]['id']
response = admin.show_search(search_id)
pprint(response)

#### Delete Search

In [ ]:
response = admin.delete_searches(search_id)
pprint(response)

### Reviews

#### List Reviews

In [ ]:
reviews = admin.list_reviews({})
pprint(reviews)

#### Show Review

In [ ]:
review_id = reviews[0]['id']
response = admin.show_review(review_id)
pprint(response)

#### Edit Review

In [ ]:
response = admin.edit_review(review_id, "New Title2", "New Description")
pprint(response)

#### Archive Review

In [ ]:
response = admin.archive_review(review_id)
pprint(response)

#### Unarchive Review

In [ ]:
response = admin.unarchive_review(review_id)
pprint(response)

#### Delete Review

In [ ]:
response = admin.delete_review(review_id)
pprint(response)

### Teams

#### List Teams

In [ ]:
teams = admin.list_teams()
pprint(teams)

#### Show Team

In [ ]:
team_id = teams[0]['id']
response = admin.show_team(team_id)
pprint(response)

#### Edit Team

In [ ]:
response = admin.edit_team(team_id, "New Team Name lol", 10, None, None, None, None, None, None)
pprint(response)

#### Create Team

In [ ]:
response = admin.create_team("New Team", 1000)
pprint(response)

#### Delete Team

In [ ]:
response = admin.delete_team(team_id)
pprint(response)